In [1]:
!python -m pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /private/var/folders/sp/4_wvps0n3nn02l9nbc4d0c7w0000gn/T/pip-req-build-y5kpz332
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/sp/4_wvps0n3nn02l9nbc4d0c7w0000gn/T/pip-req-build-y5kpz332
  Resolved https://github.com/huggingface/transformers to commit 62db3e6ed67a74cc1ed1436acd9973915c0a4475
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:

from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [2]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#model = Qwen2VLForConditionalGeneration.from_pretrained(
 #   "/Users/prasanthnandanuru/.cache/lm-studio/models/lmstudio-community/Qwen2-VL-2B-Instruct-GGUF",
  #  torch_dtype="auto",
   # device_map="auto",
#)

#processor = AutoProcessor.from_pretrained(
 #   "/Users/prasanthnandanuru/.cache/lm-studio/models/lmstudio-community/Qwen2-VL-2B-Instruct-GGUF"
#)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /Users/prasanthnandanuru/.cache/lm-studio/models/lmstudio-community/Qwen2-VL-2B-Instruct-GGUF.

In [3]:
from PIL import Image
import requests

In [6]:
image = Image.open("/Users/prasanthnandanuru/AI/Qwen2-VL-for-OCR-VQA/Simple-Invoice-Template.jpg")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": "What is total estimate in image?"
            }
        ]
    }
]

text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

inputs = processor(
    text = [text_prompt],
    images = [image],
    padding = True,
    return_tensors = "pt"
)

inputs = inputs.to("mps")

output_ids = model.generate(**inputs, max_new_tokens=1024)

generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [9]:
print(output_text)

['The total estimate in the image is $6,950.00.']


In [10]:

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": "What is unit cost of Website maintenance  in image?"
            }
        ]
    }
]

text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

inputs = processor(
    text = [text_prompt],
    images = [image],
    padding = True,
    return_tensors = "pt"
)

inputs = inputs.to("mps")

output_ids = model.generate(**inputs, max_new_tokens=1024)

generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [11]:
output_text

['The unit cost of Website maintenance is $50.00 per hour.']

In [10]:
image = Image.open("/Users/prasanthnandanuru/AI/Qwen2-VL-for-OCR-VQA/Customised_Summary_Sig.png")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": "how many hand written signatures are there in image ?"
            }
        ]
    }
]

text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

inputs = processor(
    text = [text_prompt],
    images = [image],
    padding = True,
    return_tensors = "pt"
)

inputs = inputs.to("mps")

output_ids = model.generate(**inputs, max_new_tokens=1024)

generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)


RuntimeError: MPS backend out of memory (MPS allocated: 12.09 GB, other allocations: 3.95 MB, max allowed: 18.13 GB). Tried to allocate 7.00 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [12]:
import sqlite3
conn = sqlite3.connect('tview_db.db')
cursor = conn.cursor()
cursor.execute("""
            CREATE TABLE IF NOT EXISTS txn (
                i_entity INTEGER,
                i_teller TEXT,
                n_cash_box INTEGER,
                d_business DATE,
                c_txn_type INTEGER,
                n_branch_au TEXT,
                a_start_cash INTEGER,
                a_end_cash INTEGER
            )
        """)

conn.commit()  # Important to save the table creation
print(f"Database 'tview_db.db' db and table 'txn' created successfully.")

Database 'tview_db.db' db and table 'txn' created successfully.


In [16]:
import sqlite3
conn = sqlite3.connect('tview_db.db')
cursor = conn.cursor()
cursor.execute("""
            CREATE TABLE IF NOT EXISTS metadata (
            target_type TEXT CHECK (target_type IN ('table', 'column')),
            target_name TEXT,
            metadata_key TEXT,
            metadata_value TEXT,
            PRIMARY KEY (target_type, target_name, metadata_key)
            )
        """)